In [0]:
%sh

git clone https://github.com/sjster/mlflow_dashboard.git

fatal: destination path 'mlflow_dashboard' already exists and is not an empty directory.


In [0]:
pip install -r mlflow_dashboard/requirements.txt

Python interpreter will be restarted.
  Using cached bokeh-2.4.1-py3-none-any.whl (18.5 MB)
  Using cached flair-0.10-py3-none-any.whl (322 kB)
  Using cached metaflow-2.5.2-py2.py3-none-any.whl (713 kB)
  Using cached networkx-2.6.3-py3-none-any.whl (1.9 MB)
  Using cached numpy-1.21.2-cp38-cp38-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (15.8 MB)
  Using cached pandas-1.3.5-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (11.5 MB)
  Using cached plotly-5.1.0-py2.py3-none-any.whl (20.6 MB)
  Using cached python_dateutil-2.8.2-py2.py3-none-any.whl (247 kB)
  Using cached requests-2.27.1-py2.py3-none-any.whl (63 kB)
  Using cached streamlit-1.5.1-py2.py3-none-any.whl (9.7 MB)
  Using cached PyYAML-6.0-cp38-cp38-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (701 kB)
  Using cached typing_extensions-4.1.1-py3-none-any.whl (26 kB)
  Using cached segtok-1.5.11-py3-none-any.whl (24 kB)
  Using cached gdown-3.12.2-py3-none-any.whl
  Using c

In [0]:
%sh
cd mlflow_dashboard
USERNAME='srijith.rajamohan@databricks.com' python workflow.py run --infra databricks --credentials '/dbfs/Users/srijith.rajamohan@databricks.com/PAT'

Metaflow 2.5.2 executing ForeachFlow for user:srijith.rajamohan@databricks.com
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
2022-03-30 18:03:09.841 Workflow starting (run-id 1648663389838081):
2022-03-30 18:03:09.853 [1648663389838081/start/1 (pid 7096)] Task is starting.
2022-03-30 18:03:11.887 [1648663389838081/start/1 (pid 7096)] Starting workflow on databricks with credentials from /dbfs/Users/srijith.rajamohan@databricks.com/PAT
2022-03-30 18:03:11.887 [1648663389838081/start/1 (pid 7096)] credentials path is  /dbfs/Users/srijith.rajamohan@databricks.com/PAT
2022-03-30 18:03:11.953 [1648663389838081/start/1 (pid 7096)] credentials path is  /dbfs/Users/srijith.rajamohan@databricks.com/PAT
2022-03-30 18:03:12.146 [1648663389838081/start/1 (pid 7096)] Request status  200
2022-03-30 18:03:12.147 [1648663389838081/start/1 (pid 7096)] Header info  {'Server': 'GitHub.com', 'Date': 'Wed, 30 Mar 2022 18:01:24 GMT', 'Content-Type': 'application/js

In [0]:
ls /dbfs/Users/srijith.rajamohan@databricks.com/mlflow_rundata/

comment_first_response.json*  processed_issues.json*
contributors.json*            processed_issues_entities.json*
nounphrase_dict.json*         verbphrase_dict.json*


In [0]:
dbutils.fs.cp('file:/databricks/driver/mlflow_dashboard/data/processed_issues.json','dbfs:/Users/srijith.rajamohan@databricks.com/mlflow_rundata/processed_issues.json')
dbutils.fs.cp('file:/databricks/driver/mlflow_dashboard/data/processed_issues_entities.json','dbfs:/Users/srijith.rajamohan@databricks.com/mlflow_rundata/processed_issues_entities.json')
dbutils.fs.cp('file:/databricks/driver/mlflow_dashboard/data/verbphrase_dict.json','dbfs:/Users/srijith.rajamohan@databricks.com/mlflow_rundata/verbphrase_dict.json')
dbutils.fs.cp('file:/databricks/driver/mlflow_dashboard/data/nounphrase_dict.json','dbfs:/Users/srijith.rajamohan@databricks.com/mlflow_rundata/nounphrase_dict.json')
dbutils.fs.cp('file:/databricks/driver/mlflow_dashboard/data/contributors.json','dbfs:/Users/srijith.rajamohan@databricks.com/mlflow_rundata/contributors.json')
dbutils.fs.cp('file:/databricks/driver/mlflow_dashboard/data/comment_first_response.json','dbfs:/Users/srijith.rajamohan@databricks.com/mlflow_rundata/comment_first_response.json')

Out[3]: True

In [0]:
import pandas as pd
import json

#------------------------- Processed issues ------------------------#
processed_issues = spark.read.option('multiline','true').json('dbfs:/Users/srijith.rajamohan@databricks.com/mlflow_rundata/processed_issues.json')
processed_issues.write.format('delta').mode('overwrite').saveAsTable('mlflow_processed_issues')

#------------------------- Processed issues with entities ------------------------#
processed_issues_entities = spark.read.option('multiline','true').json('dbfs:/Users/srijith.rajamohan@databricks.com/mlflow_rundata/processed_issues_entities.json')
processed_issues_entities.write.format('delta').mode('overwrite').saveAsTable('mlflow_processed_issues_entities')

#------------------------- Noun phrases ------------------------#
with open('/databricks/driver/mlflow_dashboard/data/nounphrase_dict.json','r') as f:
        nounphrases = json.load(f)
        pd.DataFrame.from_dict(nounphrases, orient='index').to_csv('/dbfs/Users/srijith.rajamohan@databricks.com/n_csv.csv', header=None)
      
temp = spark.read.csv('dbfs:/Users/srijith.rajamohan@databricks.com/n_csv.csv', header=None)
temp.write.format('delta').mode('overwrite').saveAsTable('mlflow_nounphrases_dict')
      
#------------------------- Verb phrases ------------------------#
with open('/databricks/driver/mlflow_dashboard/data/verbphrase_dict.json','r') as f:
        verbphrases = json.load(f)
        pd.DataFrame.from_dict(verbphrases, orient='index').to_csv('/dbfs/Users/srijith.rajamohan@databricks.com/v_csv.csv', header=None)
      
temp = spark.read.csv('dbfs:/Users/srijith.rajamohan@databricks.com/v_csv.csv', header=None)
temp.write.format('delta').mode('overwrite').saveAsTable('mlflow_verbphrases_dict')   

#------------------------- Metadata ------------------------#
with open('/databricks/driver/mlflow_dashboard/data/mlflow_metadata.json','r') as f:
        verbphrases = json.load(f)
        pd.DataFrame.from_dict(verbphrases, orient='index').to_csv('/dbfs/Users/srijith.rajamohan@databricks.com/mlflow_metadata.csv', header=None)
      
temp = spark.read.csv('dbfs:/Users/srijith.rajamohan@databricks.com/mlflow_metadata.csv', header=None)
temp.write.format('delta').mode('overwrite').saveAsTable('mlflow_mlflow_metadata')   

#------------------------- Contributors ------------------------#
contributors = spark.read.option('multiline','true').json('dbfs:/Users/srijith.rajamohan@databricks.com/mlflow_rundata/contributors.json')
contributors.write.format('delta').mode('overwrite').saveAsTable('mlflow_contributors')

# ------------------------ Comment time to first response ---------------------- #
comment_time = spark.read.option('multiline','true').json('dbfs:/Users/srijith.rajamohan@databricks.com/mlflow_rundata/comment_first_response.json')
comment_time.write.format('delta').mode('overwrite').saveAsTable('mlflow_comment_first_response')

In [0]:
%sql

DESCRIBE HISTORY mlflow_nounphrases_dict

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
1,2022-03-28T22:19:58.000+0000,5032213768932587,srijith.rajamohan@databricks.com,CREATE OR REPLACE TABLE AS SELECT,"Map(isManaged -> true, description -> null, partitionBy -> [], properties -> {})",null,List(3326095810992482),0929-233806-axe216,0,WriteSerializable,false,"Map(numFiles -> 1, numOutputRows -> 6, numOutputBytes -> 861)",null,Databricks-Runtime/10.4.x-photon-scala2.12
0,2022-03-28T22:04:07.000+0000,5032213768932587,srijith.rajamohan@databricks.com,CREATE OR REPLACE TABLE AS SELECT,"Map(isManaged -> true, description -> null, partitionBy -> [], properties -> {})",null,List(3326095810992482),0929-233806-axe216,null,WriteSerializable,false,"Map(numFiles -> 1, numOutputRows -> 6, numOutputBytes -> 861)",null,Databricks-Runtime/10.4.x-photon-scala2.12
